<a href="https://colab.research.google.com/github/yakoobbaba478/ML_Project/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imported all the required libraries
import os
import sys
import cv2
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
import keras.backend as K
from tqdm.autonotebook import tqdm
from os import listdir
from os.path import join,splitext,basename
from imgaug import augmenters as iaa
import random
import traceback
from shutil import rmtree
import argparse
import pandas as pd
import json
import csv
from collections import Counter
from keras.callbacks import Callback
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from IPython.display import Image
import warnings
# Keras Core
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Dropout, Dense, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras import regularizers
from keras import initializers
# Utils
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from IPython.display import Image, display

In [35]:
#mount google drive on notebook
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#extract files from Image_2.zip
# extract the .zip file. remove old directory/files if exists. 
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
if(os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Image_1')):
    rmtree('/content/gdrive/My Drive/Colab Notebooks/Image_1')
!unzip -q Image_1.zip -d Image_1

In [0]:
#extract files from Data.zip
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1')
if(os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data')):
    rmtree('/content/gdrive/My Drive/Colab Notebooks//Image_1/Image_1/Data')
!unzip -q Data.zip

In [0]:
#classify given data according to the frequency of occurence.
class_freq = {"1":[],"2":[],"3":[],"4":[],"5":[],"6":[]}  

def create_json(filename,dest_path):
    """
    Creates json files for each image
    Args:
    	filename: csv path which contains label information of each image
    	dest_path: destination path to store json files.
  
    """
    # initializing the titles and rows list 
    fields = [] 
    rows = []
    

    # reading csv file 
    with open(filename, 'r') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)       
        # extracting field names through first row 
        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 
        # get total number of rows 
        print("Total no. of rows: ",len(rows)) 

    fields = rows[0]
    rows.remove(fields)
    # printing the field names 
    print('Field names are:',fields ) 
    #  printing first 5 rows 
    print('\nFirst 5 rows are:\n') 
    large = 0
    arr = {}
    data = {}
    for row in rows: 
        col0 = str(row[0])
        image_id = int(row[1])
        col0 = col0.strip('][')
        col0 = col0.replace("u","").replace("'","").replace(" ","")
        col0 = col0.split(",")
        col0 = [int(i) for i in col0]
        for item_id in col0:
            if item_id not in arr:
                arr[item_id] = 0
            arr[item_id] += 1
        data[image_id] =  col0
        if not os.path.exists(dest_path):
            os.makedirs(dest_path)   
        with open(os.path.join(dest_path, str(image_id)+'.json'), 'w') as f:
            json.dump(data[image_id], f)   

    for i in arr.keys():
        if arr[i]<=50:
            class_freq["1"].append(i)
        elif arr[i]<150:
            class_freq["2"].append(i)
        elif  arr[i]<500:
            class_freq["3"].append(i)
        elif arr[i]<1500:
            class_freq["4"].append(i)
        elif arr[i]<3000:
            class_freq["5"].append(i)
        else:
            class_freq["6"].append(i)
            


        
        

            

In [0]:
# csv file path,destination path
filename = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train_labels.csv'
dest_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train'
create_json(filename,dest_path)
filename = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/validation_labels.csv'
dest_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/val'
create_json(filename,dest_path)

In [0]:
#compare image data with json files and remove jsons without images.
train_img_path = "/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/train"
train_json_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train'
val_img_path = "/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/validation"
val_json_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/val'
train_json_list = os.listdir(train_json_path)
val_json_list = os.listdir(val_json_path)

for i in train_json_list:
    base_name = os.path.splitext(i)[0]

    if os.path.exists(os.path.join(train_img_path,base_name+'.jpg')):
        pass
    else:
        os.remove(os.path.join(train_json_path,base_name+'.json'))
#         print("removed: ",os.path.join(train_json_path,base_name+'.json'))
for i in val_json_list:
    base_name = os.path.splitext(i)[0]

    if os.path.exists(os.path.join(val_img_path,base_name+'.jpg')):
        pass
    else:
        os.remove(os.path.join(val_json_path,base_name+'.json'))
#         print("removed: ",os.path.join(val_json_path,base_name+'.json'))

In [0]:
#apply augmentations according to their class frequency. If class freq is less apply more augmentations.
prob_class={"1":0.8,"2":0.75,"3":0.5,"4":0.4,"5":0.2,"6":0.05}
def apply_img_aug(image_folder,json_folder,class_freq):
    """
    Augmentations
    Args:
    	image_folder: Image data path
    	json_folder: json files path
        class_freq: class frequencies
    
    """

        image_files = os.listdir(image_folder)
        json_files = os.listdir(json_folder)
        pbar = tqdm(total=len(image_files))


        for an_image in image_files:
                pbar.update(1)
                
                labels = 0
                try:
                    count = 0
                    image_name_split = os.path.splitext(an_image)
                    extension_name = image_name_split[1]
                    base_name = image_name_split[0]
                    with open(os.path.join(json_folder,base_name+".json"), 'r') as f:
                        labels = json.load(f)
                    #determine probability based on occurence of classes in current image.
                    for label in labels:
                        if label in class_freq["1"]:
                            count+=1
                        elif label in class_freq["2"]:
                            count+=2
                        elif label in class_freq["3"]:
                            count+=3
                        elif label in class_freq["4"]:
                            count+=4
                        elif label in class_freq["5"]:
                            count+=5
                        elif label in class_freq["6"]:
                            count+=6
                    res = round(count/len(labels))
                    prob = prob_class[str(res)]
                    
                    
                    
                    
                    image_path = os.path.join(image_folder, an_image)

                    img_file = cv2.imread(image_path)
                    h, w, c = img_file.shape
                    
                    #Add noise to the image
                    seq_noise = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=0.10*255)])
                    #multiply image pixels(changes brightness)
                    seq_mul = iaa.Sequential([iaa.Multiply(0.5)])
                    #flip image left to right
                    seq_fliplr = iaa.Sequential([iaa.Fliplr()])
                    #add value to image pixels
                    seq_add = iaa.Sequential([iaa.Add(value=10)])
                    #rotate image
                    seq_rotate = iaa.Sequential([iaa.Rot90(1)])
                    
                    if(random.random()<prob):
                        transformed_img = seq_noise.augment_image(img_file)
                        transformed_image_base = base_name + '_noise'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                    if(random.random()<prob):          
                        transformed_img = seq_mul.augment_image(img_file)
                        transformed_image_base = base_name + '_mul'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base  +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                    if(random.random()<prob):
                        transformed_img = seq_fliplr.augment_image(img_file)
                        transformed_image_base = base_name + '_fliplr'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base  +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                    if(random.random()<prob):                    
                        transformed_img = seq_add.augment_image(img_file)
                        transformed_image_base = base_name + '_add'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base  +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                    #crop image
                    if(random.random()<prob):
                        h5,w5 = int(h*0.05),int(w*0.05)
                        transformed_img = img_file[h5:h-h5,w5:w-w5]
                        transformed_image_base = base_name + '_crop'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base  +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                    if(random.random()<prob):
                        transformed_img = seq_rotate.augment_image(img_file)
                        transformed_image_base = base_name + '_rot'
                        transformed_image_path = os.path.join(image_folder, transformed_image_base  +extension_name)
                        data = json.load(open(os.path.join(json_folder,base_name+'.json')))
                        json.dump(data, open(os.path.join(json_folder,transformed_image_base+'.json'), "w"))
                        img = cv2.cvtColor(transformed_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(transformed_image_path, img)
                except Exception as e:
                    traceback.print_exc()
                    pass
        pbar.close()


In [0]:
#image_folder path, json_folder path
image_folder = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/train'
json_folder = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train'
apply_img_aug(image_folder,json_folder,class_freq)

In [0]:
try:
    from os import scandir
except ImportError:
    from scandir import scandir
def scantree(path):
    """Recursively yield DirEntry objects for given directory."""
    for entry in scandir(path):
        if entry.is_dir(follow_symlinks=False):
            yield from scantree(entry.path)  # see below for Python 2.x
        else:
            yield entry
    
    
data = {}
json_folder = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train'

pbar = tqdm(total=50000)
for entry in scantree(json_folder):
    pbar.update(1)
    with open(os.path.join(json_folder,entry), 'r') as f:
        labels = json.load(f)
        for l in labels:
            if l not in data:
                data[l] = 0
            data[l] +=1
pbar.close()
for i in sorted(data.keys()):
    print (i,"---",data[i])
    
          

In [0]:
#reference:https://github.com/kentsommer/keras-inceptionV4
def conv2d_bn(x, nb_filter, num_row, num_col,
              padding='same', strides=(1, 1), use_bias=False):
    """
    Utility function to apply conv + BN. 
    (Slightly modified from https://github.com/fchollet/keras/blob/master/keras/applications/inception_v3.py)
    """
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    x = Convolution2D(nb_filter, (num_row, num_col),
                      strides=strides,
                      padding=padding,
                      use_bias=use_bias,
                      kernel_regularizer=regularizers.l2(0.00004),
                      kernel_initializer=initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None))(x)
    x = BatchNormalization(axis=channel_axis, momentum=0.9997, scale=False)(x)
    x = Activation('relu')(x)
    return x


In [0]:

def block_inception_a(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    branch_0 = conv2d_bn(input, 96, 1, 1)

    branch_1 = conv2d_bn(input, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3)

    branch_2 = conv2d_bn(input, 64, 1, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 96, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

In [0]:

def block_reduction_a(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    branch_0 = conv2d_bn(input, 384, 3, 3, strides=(2,2), padding='valid')

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 3, 3)
    branch_1 = conv2d_bn(branch_1, 256, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

In [0]:

def block_inception_b(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    branch_0 = conv2d_bn(input, 384, 1, 1)

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 1, 7)
    branch_1 = conv2d_bn(branch_1, 256, 7, 1)

    branch_2 = conv2d_bn(input, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 192, 7, 1)
    branch_2 = conv2d_bn(branch_2, 224, 1, 7)
    branch_2 = conv2d_bn(branch_2, 224, 7, 1)
    branch_2 = conv2d_bn(branch_2, 256, 1, 7)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 128, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

In [0]:
def block_reduction_b(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    branch_0 = conv2d_bn(input, 192, 1, 1)
    branch_0 = conv2d_bn(branch_0, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_1 = conv2d_bn(input, 256, 1, 1)
    branch_1 = conv2d_bn(branch_1, 256, 1, 7)
    branch_1 = conv2d_bn(branch_1, 320, 7, 1)
    branch_1 = conv2d_bn(branch_1, 320, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

In [0]:

def block_inception_c(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    branch_0 = conv2d_bn(input, 256, 1, 1)

    branch_1 = conv2d_bn(input, 384, 1, 1)
    branch_10 = conv2d_bn(branch_1, 256, 1, 3)
    branch_11 = conv2d_bn(branch_1, 256, 3, 1)
    branch_1 = concatenate([branch_10, branch_11], axis=channel_axis)


    branch_2 = conv2d_bn(input, 384, 1, 1)
    branch_2 = conv2d_bn(branch_2, 448, 3, 1)
    branch_2 = conv2d_bn(branch_2, 512, 1, 3)
    branch_20 = conv2d_bn(branch_2, 256, 1, 3)
    branch_21 = conv2d_bn(branch_2, 256, 3, 1)
    branch_2 = concatenate([branch_20, branch_21], axis=channel_axis)

    branch_3 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 256, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

In [0]:
#Network architecture.
def inception_v4_base(input):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    # Input Shape is 299 x 299 x 3 (th) or 3 x 299 x 299 (th)
    net = conv2d_bn(input, 32, 3, 3, strides=(2,2), padding='valid')
    net = conv2d_bn(net, 32, 3, 3, padding='valid')
    net = conv2d_bn(net, 64, 3, 3)

    branch_0 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    branch_1 = conv2d_bn(net, 96, 3, 3, strides=(2,2), padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 64, 1, 1)
    branch_0 = conv2d_bn(branch_0, 96, 3, 3, padding='valid')

    branch_1 = conv2d_bn(net, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 64, 1, 7)
    branch_1 = conv2d_bn(branch_1, 64, 7, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3, padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 192, 3, 3, strides=(2,2), padding='valid')
    branch_1 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    # 35 x 35 x 384
    # 4 x Inception-A blocks
    for idx in range(4):
    	net = block_inception_a(net)

    # 35 x 35 x 384
    # Reduction-A block
    net = block_reduction_a(net)

    # 17 x 17 x 1024
    # 7 x Inception-B blocks
    for idx in range(7):
    	net = block_inception_b(net)

    # 17 x 17 x 1024
    # Reduction-B block
    net = block_reduction_b(net)

    # 8 x 8 x 1536
    # 3 x Inception-C blocks
    for idx in range(3):
    	net = block_inception_c(net)

    return net

In [0]:
global model

def inception_v4(num_classes, dropout_keep_prob, weights, include_top):
    '''
    Creates the inception v4 network
    Args:
    	num_classes: number of classes
    	dropout_keep_prob: float, the fraction to keep before final layer.
    
    Returns: 
    	logits: the logits outputs of the model.
    '''

    # Input Shape is 299 x 299 x 3 (tf) or 3 x 299 x 299 (th)
    if K.image_data_format() == 'channels_first':
        inputs = Input((3, 299, 299))
    else:
        inputs = Input((299, 299, 3))

    # Make inception base
    x = inception_v4_base(inputs)


    # Final pooling and prediction
    if include_top:
        # 1 x 1 x 1536
        x = AveragePooling2D((8,8), padding='valid')(x)
        x = Dropout(dropout_keep_prob)(x)
        x = Flatten()(x)
        # 1536
        x = Dense(units=num_classes, activation='sigmoid')(x)

    model = Model(inputs, x, name='inception_v4')
    return model

In [0]:
def create_model(num_classes=228, dropout_prob=0.2, weights=None, include_top=True):
    return inception_v4(num_classes, dropout_prob, weights, include_top)
model = create_model()

In [0]:
#define custom loss function.

import functools
def prn_loss_cls(y_true, y_pred):
       return K.sum(K.binary_crossentropy(y_true,y_pred), axis=-1)
    
top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'
optim = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optim, loss=prn_loss_cls, metrics=['acc',top5_acc])

In [0]:

def get_input(path):
    try:
        img = cv2.imread(path)
        return(img)
    except:
        return None
def get_output(path,label_dir):
    try:
        labels = ()
        ground_truth = np.zeros(229)
        img_id = os.path.splitext(os.path.basename(path))[0]
        json_path = os.path.join(label_dir,str(img_id)+".json")
        with open(json_path, 'r') as f:
            labels = json.load(f)
        #print(labels) 
        ground_truth[labels]+=1
        ground_truth = ground_truth[1:]
        #print("lables: ",labels)
        return(ground_truth)
    except:
        return None
def preprocess_input(x=None):
    input_img = x/255
    input_img = cv2.resize(input_img,(299, 299))
    return input_img


def image_generator(img_list,img_dir,label_dir, batch_size = 8):
    """
    ImageDataGenerator
    parameters
        img_list: list of image data
        img_dir: image data directory
        label_dir: json files directory
        batch_size: batch size
    """
    
    while True:          
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = img_list, 
                                        size = batch_size+2)
        batch_input = []
        batch_output = [] 
        count = 0
        # Read in each input, perform preprocessing and get labels          
        for input_path in batch_paths:              
            input_img = get_input(os.path.join(img_dir,input_path) )
            if input_img is None:
                continue
            ground_truth = get_output(input_path,label_dir )
            if ground_truth is None:
                continue
            input_img = preprocess_input(x=input_img)
            #print(input_img.shape)
            batch_input.append(input_img)
            batch_output.append(ground_truth)         
        # Return a tuple of (input,output) to feed the network 
        #print("batch_input: ",len(batch_input))
        #print("bath_output: ",len(batch_output))        
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        #print("batchx[0]: ",batch_x[0])
        #print("batchy[0]: ",batch_y[0])
#         print("batch_x: ",batch_x.shape)
#         print("bath_y: ",batch_y.shape)
        
        
        yield( batch_x, batch_y )
        


In [0]:
def train(train_path,label_dir, val_path, checkpoint_path):
    """train
    Parameters: 
   train_path(str): directory path to train images with subfolders as classes.
   val_path(str): directory path to validation images with subfolders as classes.
   checkpoint_path(str): directory path save checkpoints 
  """

    img_list = os.listdir(train_path)
    n = len(img_list)
    train_set = int(n*0.80)
    val_img_list = []
    train_img_list = random.sample(img_list,train_set)
    for i in img_list:
        if i not in train_img_list:
            val_img_list.append(i)
    training_set = image_generator(train_img_list,train_path,label_dir, batch_size = 8)
    validation_set = image_generator(val_img_list,train_path,label_dir,batch_size = 8)
    
    
    checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=1)
        
    callbacks_list = [checkpoint]	

    model.fit_generator(training_set, steps_per_epoch = 3000, epochs = 30,callbacks=callbacks_list, validation_data = validation_set, validation_steps = 10)
    #print ("after model.fit")

   

In [0]:
train_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/train'
label_dir = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/train'
checkpoint_dir = '/content/gdrive/My Drive/Colab Notebooks/classifier_weights2/'
val_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/validation'
checkpoint_path = '/content/gdrive/My Drive/Colab Notebooks/classifier_weights2/weights.{epoch:02d}_{val_acc:.3f}.h5'
train(train_path,label_dir,val_path,checkpoint_path)

In [0]:

os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/train')

In [0]:
#selecting best model from classifier_directory
#Loading the best model
checkpoint_dir = '/content/gdrive/My Drive/Colab Notebooks/classifier_weights2/'
model_list = os.listdir(checkpoint_dir)
print(model_list)
best_model_path = ''
best_score = 0.0
for m in model_list:
    m_name = os.path.splitext(m)[0]
    m_acc = float(m_name.split('_')[1])
    if(m_acc>best_score):
        best_model_path = m
        best_score = m_acc
        
best_model = os.path.join(checkpoint_dir,best_model_path)
model.load_weights(best_model)
print("best model: ",best_model)

In [0]:
#evaluate best model on test set
test_path = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/validation'
label_dir = '/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Input/val'
test_img_list = os.listdir(test_path)
print(os.path.exists('/content/gdrive/My Drive/Colab Notebooks/Image_1/Image_1/Data/validation'))
print("testing started........",len(test_img_list))

p_bar = tqdm(total=len(test_img_list))
print (len(test_img_list[:800]))
for i in random.sample(test_img_list[:2000],800):
    p_bar.update(1)
    correct = 0
    precisions = []
    ground_truth = []
    img_path = os.path.join(test_path,i)
    name = os.path.basename(img_path)[0]
    test_img = cv2.imread(img_path)
    test_img = test_img/255
    test_img = cv2.resize(test_img, (299, 299))
    test_img = np.array([test_img])
    json_path = os.path.join(label_dir,name+'.json')
    with open(json_path, 'r') as f:
        ground_truth = json.load(f)
    ground_truth = np.array(ground_truth)
#     print(name,"----->",ground_truth)
    output = model.predict(test_img)
#     print (output)
#     output = np.nonzero(output > 0.5)
    output = np.where(output>=0.5)
    output = output[1]
    output = [i+1 for i in output]
#     print ("output:",sorted(output))
#     print ("ground:",sorted(ground_truth))
#     display(Image(filename=img_path))
#     break
    for i in output:
        if i in ground_truth:
            correct += 1
    precision = correct/len(ground_truth)
    precisions.append(precision)
    
p_bar.close()
#     print("output--->",output)
#     print(precisions)
average_precision = sum(precisions)/len(precisions)
print("average_precision: ",average_precision)
print("testing ended............")


# confusion_matrix = np.zeros((4,4))
# p_bar = tqdm(total=len(test_img_list))
# for idx in range(len(test_img_list)):
#     x = test_set[idx]
#     p_bar.update(1)
#     test_image = x[0]
#     test_image_label = x[1]
#     print(test_image,"----->",test_image_label)
#     truth = np.argmax(test_image_label)
#     output = model.predict(test_image)
#     print("output---->",output)
#     break
#     output_idx = np.argmax(output)
#     confusion_matrix[truth][output_idx]+=1
# p_bar.close()
    
# print("confusion matrix:")
# print(confusion_matrix)